In [1]:
# uncomment and run the lines below if running in google colab
!pip install tensorflow==2.4.3
!git clone https://github.com/Skand21/SRGAN.git # here is images
!git clone https://github.com/jlaihong/image-super-resolution.git
!mv image-super-resolution/* ./

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.5 MB 356 bytes/s 
     |████████████████████████████████| 132 kB 29.9 MB/s 
     |████████████████████████████████| 3.8 MB 42.7 MB/s 
     |████████████████████████████████| 462 kB 71.3 MB/s 
     |████████████████████████████████| 2.9 MB 40.1 MB/s 
     |████████████████████████████████| 14.8 MB 50.9 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68722 sha256=b704afa0fa1af0ba26c46939afa5b29e056392b0e9d11e8387b014dc810eb0ce
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: numpy
    Found existing installatio

Cloning into 'SRGAN'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 93 (delta 15), reused 92 (delta 14), pack-reused 0
Unpacking objects: 100% (93/93), done.
Cloning into 'image-super-resolution'...
remote: Enumerating objects: 150, done.
remote: Total 150 (delta 0), reused 0 (delta 0), pack-reused 150
Receiving objects: 100% (150/150), 36.74 MiB | 29.60 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [2]:
import os
import glob
import numpy as np
from google.colab import drive
from google.colab import files
from PIL import Image, ImageOps
import tensorflow as tf

from datasets.div2k.parameters import Div2kParameters 
from models.srresnet import build_srresnet
from models.pretrained import pretrained_models
from utils.prediction import get_sr_image
from utils.config import config

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
%cd ../content/input
uploaded = files.upload()
nameFile = ""
for fn in uploaded.keys():
  if nameFile == "":
    nameFile = fn
%cd ..
# from google.colab import drive
# drive.mount('/content/drive')

/content/input


Saving я у доски.jpg to я у доски (1).jpg
/content


In [4]:
dataset_key = "bicubic_x4"

data_path = config.get("data_path", "") 

div2k_folder = os.path.abspath(os.path.join(data_path, "div2k"))

dataset_parameters = Div2kParameters(dataset_key, save_data_directory=div2k_folder)

In [5]:
def load_image(path):
    img = Image.open(path)
    
    was_grayscale = len(img.getbands()) == 1
    
    if was_grayscale or len(img.getbands()) == 4:
        img = img.convert('RGB')
    
    return was_grayscale, np.array(img)


In [6]:
# model_name = "srresnet"
model_name = "srgan"

In [7]:
model_key = f"{model_name}_{dataset_key}"

In [8]:
weights_directory = os.path.abspath(f"weights/{model_key}")

file_path = os.path.join(weights_directory, "generator.h5")

if not os.path.exists(file_path):
    os.makedirs(weights_directory, exist_ok=True)
    
    print("Couldn't find file: ", file_path, ", attempting to download a pretrained model")
    
    if model_key not in pretrained_models:
        print(f"Couldn't find pretrained model with key: {model_key}, available pretrained models: {pretrained_models.key()}")
    else:
        download_url = pretrained_models[model_key]
        file = file_path.split("/")[-1]
        tf.keras.utils.get_file(file, download_url, cache_subdir=weights_directory)

Couldn't find file:  /content/weights/srgan_bicubic_x4/generator.h5 , attempting to download a pretrained model
6529024/6521184 [==============================] - 2s 0us/step


In [9]:
results_path = f"output/{model_key}/"

In [10]:
model = build_srresnet(scale=dataset_parameters.scale)

os.makedirs(weights_directory, exist_ok=True)
weights_file = f'{weights_directory}/generator.h5'

model.load_weights(weights_file)

In [11]:
for file in glob.glob("output/srgan_bicubic_x4/*"):
    os.remove(file)
os.makedirs(results_path, exist_ok=True)

In [12]:
was_grayscale, lr = load_image("input/" + nameFile)
    
sr = get_sr_image(model, lr)
    
if was_grayscale:
    sr = ImageOps.grayscale(sr)

sr.save(f"{results_path}{nameFile}" )

In [13]:
os.rename("output/srgan_bicubic_x4/" + nameFile, "output/srgan_bicubic_x4/image.jpg")
files.download("output/srgan_bicubic_x4/image.jpg")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>